<a href="https://colab.research.google.com/github/IlyaBoss2414/IPEC/blob/main/IPEC_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install rdkit-pypi

In [5]:
import rdkit
import numpy as np
from rdkit.Chem import MACCSkeys, AllChem, DataStructs
from rdkit.Avalon.pyAvalonTools import GetAvalonFP
from rdkit import Chem
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
from rdkit.Chem import rdFMCS
from rdkit.Chem.Draw import IPythonConsole
import pickle
import os
from matplotlib import pyplot as plt
from matplotlib.image import NonUniformImage
import numpy as np
from sys import getsizeof
import psutil
import torch


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import RocCurveDisplay
try:
    from catboost import CatBoostClassifier
except:
    !pip install catboost
    from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
try:
    import optuna
except:
    !pip install optuna
    import optuna

In [11]:
import numpy as np
import random
import torch

def set_random_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

set_random_seed(42)

In [7]:
# Load data and shuffle before splitting
path_to_folder = "/content/drive/MyDrive/MsuAI/Project/Data"
df = pd.read_csv(os.path.join(path_to_folder, 'df_lin.csv')).sample(frac=1, random_state = 27)
df = df.drop(columns=['Unnamed: 0'])


In [8]:
# best features obtained using the rdkit library and selected using combubed Boruta, Boruta, built-in feature extraction approaches, Lasso, mRmR
best_rdkit_features = ['PA_BalabanJ',
 'PС_fr_Ndealkylation1',
 'PA_EState_VSA7',
 'PС_SlogP_VSA4',
 'PС_SMR_VSA4',
 'PA_SMR_VSA5',
 'PС_fr_unbrch_alkane',
 'PA_SPS',
 'PA_VSA_EState8',
 'PС_qed']

my_features = ["PA_DP", "PC_DP", "NaCl", "PA_idx", "PC_idx","eps","eps1","eps2"]

list_features = my_features + best_rdkit_features


In [9]:
df_test = pd.concat([df.loc[(df["PA_idx"] == 1) & (df["PC_idx"] == 3)].copy(), df.loc[(df["PA_idx"] == 4) & (df["PC_idx"] == 2)].copy()]) # index_to_drop = df_test.index
df_train_all = df.drop(index=df_test.index)  # Sample 20% for the validation set
df_train = df_train_all.sample(frac=0.8, random_state = 42)  # Sample 20% for the test set
df_val = df_train_all.drop(index=df_train.index)  # Sample

In [19]:
# hyperparameters selected using Optuna of 5 folds by groups = df_train_all["PC_idx"]
best_params =  {'iterations': 202, 'depth': 4, 'learning_rate': 0.04420107240307016, 'l2_leaf_reg': 6.703979878946633e-05, 'border_count': 75}
final_model = CatBoostRegressor(**best_params)


X_train_val = df_train_all[list_features]
X_test = df_test[list_features]

y_train_val = df_train_all['phi'].astype(float)
y_test = df_test['phi'].astype(float)

# Step 2: Scale the features
scaler = StandardScaler()
X_train_val_scaled = scaler.fit_transform(X_train_val)
X_test_scaled = scaler.transform(X_test)

# Fit the final model
final_model.fit(X_train_val_scaled, y_train_val, early_stopping_rounds=100, verbose=False)

# Predict and evaluate on test set
y_test_pred = final_model.predict(X_test_scaled)

# Calculate metrics
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
test_mae = mean_absolute_error(y_test, y_test_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)
test_mape = mean_absolute_percentage_error(y_test, y_test_pred)

print(f'Test MAE score: {test_mae}')
print(f'Test MSE score: {test_mse}')
print(f'Test R² score: {test_r2}')
print(f'Test MAPE score: {test_mape}')



Test MAE score: 2.097793687180338
Test MSE score: 14.07965028988206
Test R² score: 0.9022997679652726
Test MAPE score: 0.045690820445396
